# Sentiment Analysis of Movie Review Data with Logistic Regression and Naive Bayes Algorithms

## Objective

The goal of this project is to perform a sentiment analysis of text data from movie reviews using two different classification algorithms - Logistic Regression and Naive Bayes - and compare their performance.

## Data Collection and Preparation

This dataset was organized and labeled by Bo Pang and Lillian Li, and used in their paper "Seeing Stars: Exploiting class relationships for sentiment categorization with respect to rating scales" (2005) from Proceedings of the ACL. It contains 5331 positive and 5331 negative review snippets culled from the Rotten Tomatoes movie review website. Positive reviews were those designated as "fresh" and negative reviews were those designated "rotten" on the website. The full data set can be accessed [here](http://www.cs.cornell.edu/people/pabo/movie-review-data)

In [1]:
# initialize empty lists to store positive and negative reviews
positive_reviews = []
negative_reviews = []

# read the positive and negative reviews into their respective lists from the data files
with open("positive_reviews.txt", "r") as positive_file:
    for line in positive_file:
        # remove leading/trailing whitespaces and add to positive_reviews
        positive_reviews.append(line.strip())

with open("negative_reviews.txt", "r") as negative_file:
    for line in negative_file:
        # remove leading/trailing whitespaces and add to positive_reviews
        negative_reviews.append(line.strip())
        
# print the first few lines from each list as a sample
print("Sample Positive Reviews: ")
print(positive_reviews[:3])
print("Sample Negative Reviews: ")
print(negative_reviews[:3])

Sample Positive Reviews: 
['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'effective but too-tepid biopic']
Sample Negative Reviews: 
['simplistic , silly and tedious .', "it's so laddish and juvenile , only teenage boys could possibly find it funny .", 'exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable .']


In [2]:
# create the x-variable
reviews = positive_reviews + negative_reviews
print(len(reviews))
# create the y-variable
labels = [1] * len(positive_reviews) + [0] * len(negative_reviews)
print(len(labels))

10662
10662


clean and tokenize the reviews

In [3]:
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
nltk.download('punkt') # pre-trained tokenizer Punkt for English
def tokenize(corpus):
    data = []
    for text in corpus: 
        text = re.sub(r'\\', ' ', text)
        text = re.sub(r'[,!?;-]+', '.', text)
        words = nltk.word_tokenize(text) #tokenize strings to words
        words = [word.lower() for word in words
                if word.isalpha()
                or word == '.'
               ]
        data.extend(words)
    return data

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vscerra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
print(reviews[:10])

['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'effective but too-tepid biopic', 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .', "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .", 'the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .', 'offers that rare combination of entertainment and education .', 'perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .', "steers turns in a snap

In [5]:
data = tokenize(reviews)

In [6]:
print(f'Sample of words: {data[:20]}')

Sample of words: ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', 'century', 'new', 'conan', 'and', 'that', 'he', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater']


now make a sliding window of words - Slide a window of words across the list. For each window, we extract the center word and surrounding context words. 

In [7]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i-C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1
        

The first argument in the `get_windows` function is the list of tokenized words, and the second argument is the context half-size. For a given center word, the context words are the `C` flanking words on either side.

In [8]:
for x, y in get_windows(tokenize(["Now try the tokenizer with a new sentence"]), 1):
    print(f'{x}\t{y}')

['now', 'the']	try
['try', 'tokenizer']	the
['the', 'with']	tokenizer
['tokenizer', 'a']	with
['with', 'new']	a
['a', 'sentence']	new


### Transform the training set reviews into vectors

We want to transfer the center words into one-hot vectors, and the context words into averages of one-hot vectors. 

To create the one-hot vectors, start by mapping each word to a unique index. The function below creates Python dictionaries that map words to integers (word2ind) and back (ind2word).

In [9]:
def word_to_ind_mapping(words):
    # create a set of unique words in the list
    unique_words = list(set(words))
    
    # create a word to index mapping (word2Ind)
    word2ind = {word: idx for idx, word in enumerate(unique_words)}
    
    # create an index to word mapping (Ind2word)
    ind2word = {idx: word for idx, word in enumerate(unique_words)}
    
    return word2ind, ind2word

In [10]:
word2ind, ind2word = word_to_ind_mapping(data)
V = len(word2ind)
print("size of vocabluary: ", V)

size of vocabluary:  17596


### Get one-hot word vectors

In [11]:
def word_to_one_hot_vector(word, word2ind, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2ind[word]] = 1
    
    return one_hot_vector

Getting the average of one-hot vectors for context words

In [12]:
def context_words_to_vector(context_words, word2ind, V):
    context_word_vectors = [word_to_one_hot_vector(w, word2ind, V) for w in context_words]
    context_word_vectors = np.mean(context_word_vectors, axis = 0)
    
    return context_word_vectors

## Train the CBOW model

### Initialize the model matrices and vectors

In [13]:
def initialize_model(N, V, random_seed = 1):
    '''
    Inputs: 
        N:  dimension of hidden vector 
        V:  dimension of vocabulary
        random_seed: random seed for consistent results during testbedding
     Outputs: 
        W1, W2, b1, b2: initialized weights and biases
    '''
    np.random.seed(random_seed)
    # W1 has shape (N,V)
    W1 = np.random.rand(N,V)
    
    # W2 has shape (V,N)
    W2 = np.random.rand(V,N)
    
    # b1 has shape (N,1)
    b1 = np.random.rand(N,1)
    
    # b2 has shape (V,1)
    b2 = np.random.rand(V,1)
    
    return W1, W2, b1, b2

In [14]:
# Test your function example.
tmp_N = 4
tmp_V = 10
tmp_W1, tmp_W2, tmp_b1, tmp_b2 = initialize_model(tmp_N,tmp_V)
assert tmp_W1.shape == ((tmp_N,tmp_V))
assert tmp_W2.shape == ((tmp_V,tmp_N))
print(f"tmp_W1.shape: {tmp_W1.shape}")
print(f"tmp_W2.shape: {tmp_W2.shape}")
print(f"tmp_b1.shape: {tmp_b1.shape}")
print(f"tmp_b2.shape: {tmp_b2.shape}")

tmp_W1.shape: (4, 10)
tmp_W2.shape: (10, 4)
tmp_b1.shape: (4, 1)
tmp_b2.shape: (10, 1)


### Implement the softmax and ReLU activation functions

In [15]:
def softmax(z):
    e_z = np.exp(z)
    sum_e_z = np.sum(e_z)
    
    return e_z / sum_e_z

In [16]:
# check the softmax function
softmax([9, 8, 11, 12, 10, 10])

array([0.02917235, 0.01073191, 0.21555612, 0.58594229, 0.07929867,
       0.07929867])

In [17]:
def relu(z): 
    result = z.copy()
    result[result < 0] = 0
    
    return result

In [18]:
# check the relu function
relu(np.array([[-1, 2, 3, -100, 10]]))

array([[ 0,  2,  3,  0, 10]])

### Implement forward propagation

Implement the forward propagation of $z$ according to the following equations. <br>

\begin{align}
 h &= W_1 \  X + b_1   \\
 h &= ReLU(h)  \\
 z &= W_2 \  h + b_2    \\
\end{align}


In [19]:
def forward_propagation(x, W1, W2, b1, b2, relu = relu):
    '''
    Inputs: 
        x:  average one hot vector for the context 
        W1, W2, b1, b2:  matrices and biases to be learned
     Outputs: 
        z:  output score vector
    '''
    # calculate h
    z1 = np.dot(W1,x) + b1
    
    # apply relu on h, store in h
    h = relu(z1)
    
    # calculate z
    z = np.dot(W2, h) + b2
    
    return z, h

In [20]:
# Test the function

# Create some inputs
tmp_N = 2
tmp_V = 3
tmp_x = np.array([[0,1,0]]).T
#print(tmp_x)
tmp_W1, tmp_W2, tmp_b1, tmp_b2 = initialize_model(N=tmp_N,V=tmp_V, random_seed=1)

print(f"x has shape {tmp_x.shape}")
print(f"N is {tmp_N} and vocabulary size V is {tmp_V}")

# call function
tmp_z, tmp_h = forward_propagation(tmp_x, tmp_W1, tmp_W2, tmp_b1, tmp_b2)
print("call forward_prop")
print()
# Look at output
print(f"z has shape {tmp_z.shape}")
print("z has values:")
print(tmp_z)

print()

print(f"h has shape {tmp_h.shape}")
print("h has values:")
print(tmp_h)

x has shape (3, 1)
N is 2 and vocabulary size V is 3
call forward_prop

z has shape (3, 1)
z has values:
[[0.55379268]
 [1.58960774]
 [1.50722933]]

h has shape (2, 1)
h has values:
[[0.92477674]
 [1.02487333]]


### Generate data batches to run through the algorithm

In [21]:
def get_batches(corpus, word2ind, V, C, batch_size):
    data = [word2ind[word] for word in corpus if word in word2ind]
    
    context_word_matrix_list = []  # Initialize a list to store context word matrices
    center_word_matrix_list = []  # Initialize a list to store center word matrices
        
    for i in range(C, len(data) - C):
        context_words = []
        center_word = data[i]
        
        # collect context words in the window
        for j in range(i-C, i):
            context_words.append(data[j])
        for j in range(i+1, i+C+1):
            context_words.append(data[j])
            
        # convert context and center words to vectors
        context_vector = context_words_to_vector([corpus[idx] for idx in context_words], word2ind, V)
        center_word_vector = word_to_one_hot_vector(corpus[center_word], word2ind, V)
        
        # append context vector and center vector to their respective lists
        context_word_matrix_list.append(context_vector)
        center_word_matrix_list.append(center_word_vector)
        
        # if the batch size is reached, yield the batch and reset it
        if len(context_word_matrix_list) == batch_size:
            yield np.column_stack(context_word_matrix_list), np.column_stack(center_word_matrix_list)
            # reset the matrix lists
            context_word_matrix_list = []
            center_word_matrix_list = []
            
    # yield any remaining batches
    if context_word_matrix_list: 
        yield  np.column_stack(context_word_matrix_list), np.column_stack(center_word_matrix_list)
    

### Implement the cross-entropy cost function

In [22]:
def compute_cost(y, yhat, batch_size): 
    # cost function
    logprobs = np.multiply(np.log(yhat),y)
    cost = -1 / batch_size * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost

In [23]:
# Test the function
tmp_C = 2
tmp_N = 50
tmp_batch_size = 4
tmp_word2Ind, tmp_Ind2word = word_to_ind_mapping(data)
tmp_V = len(word2ind)
tmp_x, tmp_y = next(get_batches(data, tmp_word2Ind, tmp_V,tmp_C, tmp_batch_size))
        
print(f"tmp_x.shape {tmp_x.shape}")
print(f"tmp_y.shape {tmp_y.shape}")

tmp_W1, tmp_W2, tmp_b1, tmp_b2 = initialize_model(tmp_N,tmp_V)

print(f"tmp_W1.shape {tmp_W1.shape}")
print(f"tmp_W2.shape {tmp_W2.shape}")
print(f"tmp_b1.shape {tmp_b1.shape}")
print(f"tmp_b2.shape {tmp_b2.shape}")

tmp_z, tmp_h = forward_propagation(tmp_x, tmp_W1, tmp_W2, tmp_b1, tmp_b2)
print(f"tmp_z.shape: {tmp_z.shape}")
print(f"tmp_h.shape: {tmp_h.shape}")

tmp_yhat = softmax(tmp_z)
print(f"tmp_yhat.shape: {tmp_yhat.shape}")

tmp_cost = compute_cost(tmp_y, tmp_yhat, tmp_batch_size)
print("call compute_cost")
print(f"tmp_cost {tmp_cost:.4f}")

tmp_x.shape (17596, 4)
tmp_y.shape (17596, 4)
tmp_W1.shape (50, 17596)
tmp_W2.shape (17596, 50)
tmp_b1.shape (50, 1)
tmp_b2.shape (17596, 1)
tmp_z.shape: (17596, 4)
tmp_h.shape: (50, 4)
tmp_yhat.shape: (17596, 4)
call compute_cost
tmp_cost 14.0589


### Implement backpropagation

In [24]:
def back_propagation(x, yhat, y, h, W1, W2, b1, b2, batch_size):
    '''
    Inputs: 
        x:  average one hot vector for the context 
        yhat: prediction (estimate of y)
        y:  target vector
        h:  hidden vector
        W1, W2, b1, b2:  matrices and biases  
        batch_size: batch size 
     Outputs: 
        grad_W1, grad_W2, grad_b1, grad_b2:  gradients of matrices and biases   
    '''
        
    # compute z1 as W1x + b1
    z1 = np.dot(W1,x) +  b1
    
    # initialize gradients
    grad_W1 = np.zeros_like(W1)
    grad_W2 = np.zeros_like(W2)
    grad_b1 = np.zeros_like(b1)
    grad_b2 = np.zeros_like(b2)
    
    for i in range(batch_size):
        x_i = x[:, i].reshape(-1, 1)
        y_i = y[:, i].reshape(-1, 1)
        yhat_i = yhat[:, i].reshape(-1,1)
        z1_i = z1[:, i].reshape(-1, 1)
        
        # compute l1 as W2^T (yhat - y)
        l1 = np.dot(W2.T, yhat_i - y_i)
        l1[z1_i < 0] = 0
        
        # use l1 to compute gradients below
        
        # compute gradient for W1
        grad_W1 += np.outer(l1, x_i)
        
        # compute gradient for W2
        grad_W2 += np.outer(yhat_i - y_i, h[:, i].reshape(-1,1))
        
        # compute gradient for b1
        grad_b1 += l1
        
        # compute graident for b2 
        grad_b2 += (yhat_i - y_i)
        
    # average the gradients over the batch
    grad_W1 /= batch_size
    grad_W2 /= batch_size
    grad_b1 /= batch_size
    grad_b2 /= batch_size
    
    return grad_W1, grad_W2, grad_b1, grad_b2


In [25]:
# Test the function
tmp_C = 2
tmp_N = 50
tmp_batch_size = 4
tmp_word2Ind, tmp_Ind2word = word_to_ind_mapping(data)
tmp_V = len(word2ind)


# get a batch of data
tmp_x, tmp_y = next(get_batches(data, tmp_word2Ind, tmp_V,tmp_C, tmp_batch_size))

print("get a batch of data")
print(f"tmp_x.shape {tmp_x.shape}")
print(f"tmp_y.shape {tmp_y.shape}")

print()
print("Initialize weights and biases")
tmp_W1, tmp_W2, tmp_b1, tmp_b2 = initialize_model(tmp_N,tmp_V)

print(f"tmp_W1.shape {tmp_W1.shape}")
print(f"tmp_W2.shape {tmp_W2.shape}")
print(f"tmp_b1.shape {tmp_b1.shape}")
print(f"tmp_b2.shape {tmp_b2.shape}")

print()
print("Forwad prop to get z and h")
tmp_z, tmp_h = forward_propagation(tmp_x, tmp_W1, tmp_W2, tmp_b1, tmp_b2)
print(f"tmp_z.shape: {tmp_z.shape}")
print(f"tmp_h.shape: {tmp_h.shape}")

print()
print("Get yhat by calling softmax")
tmp_yhat = softmax(tmp_z)
print(f"tmp_yhat.shape: {tmp_yhat.shape}")

tmp_m = (2*tmp_C)
tmp_grad_W1, tmp_grad_W2, tmp_grad_b1, tmp_grad_b2 = back_propagation(tmp_x, tmp_yhat, tmp_y, tmp_h, tmp_W1, tmp_W2, tmp_b1, tmp_b2, tmp_batch_size)

print()
print("call back_prop")
print(f"tmp_grad_W1.shape {tmp_grad_W1.shape}")
print(f"tmp_grad_W2.shape {tmp_grad_W2.shape}")
print(f"tmp_grad_b1.shape {tmp_grad_b1.shape}")
print(f"tmp_grad_b2.shape {tmp_grad_b2.shape}")

get a batch of data
tmp_x.shape (17596, 4)
tmp_y.shape (17596, 4)

Initialize weights and biases
tmp_W1.shape (50, 17596)
tmp_W2.shape (17596, 50)
tmp_b1.shape (50, 1)
tmp_b2.shape (17596, 1)

Forwad prop to get z and h
tmp_z.shape: (17596, 4)
tmp_h.shape: (50, 4)

Get yhat by calling softmax
tmp_yhat.shape: (17596, 4)

call back_prop
tmp_grad_W1.shape (50, 17596)
tmp_grad_W2.shape (17596, 50)
tmp_grad_b1.shape (50, 1)
tmp_grad_b2.shape (17596, 1)


### Batch Gradient Descent

In [33]:
def gradient_descent(data, word2ind, N, V, C, num_iters, alpha = 0.01,
                    random_seed = 1, initialize_model = initialize_model, 
                    get_batches = get_batches, forward_propagation = forward_propagation,
                    softmax = softmax, compute_cost = compute_cost,
                    back_propagation = back_propagation): 
    '''
      Inputs: 
        data:      text
        word2ind:  words to Indices
        N: dimension of hidden vector  
        V: dimension of vocabulary 
        C: size of the context around center word
        num_iters: number of iterations  
        random_seed: random seed to initialize the model's matrices and vectors
        initialize_model: your implementation of the function to initialize the model
        get_batches: function to get the data in batches
        forward_propagation: your implementation of the function to perform forward propagation
        softmax: your implementation of the softmax function
        compute_cost: cost function (Cross entropy)
        back_propagation: your implementation of the function to perform backward propagation
     Outputs: 
        W1, W2, b1, b2:  updated matrices and biases after num_iters iterations

    '''
    W1, W2, b1, b2 = initialize_model(N, V, random_seed = random_seed) 
    
    batch_size = 36
    iters = 0
    
    for x, y in get_batches(data, word2ind, V, C, batch_size):
        # get z and h
        z, h = forward_propagation(x, W1, W2, b1, b2)
        
        # get yhat
        yhat = softmax(z)
        
        # get cost
        cost = compute_cost(y, yhat, batch_size)
        if ((iters + 1) % 10 == 0):
            print(f"iters: {iters + 1} cost: {cost: .6f}")
            
        # get gradients 
        #print(x.shape, yhat.shape, y.shape, h.shape, W1.shape, W2.shape, b1.shape, b2.shape, batch_size)
        grad_W1, grad_W2, grad_b1, grad_b2 = back_propagation(x, yhat, y, h, W1, W2, b1, b2, batch_size)
        
        # update weights and biases
        W1 = W1 - alpha * grad_W1
        W2 = W2 - alpha * grad_W2
        b1 = b1 - alpha * grad_b1
        b2 = b2 - alpha * grad_b2
        
        iters += 1
        if iters == num_iters:
            break
        if iters % 100 == 0:
            alpha *= 0.66
        
    return W1, W2, b1, b2

In [ ]:
# Test the function
C = 2
N = 200
word2Ind, Ind2word = word_to_ind_mapping(data)
V = len(word2Ind)
num_iters = 1000
print("Call gradient_descent")
W1, W2, b1, b2 = gradient_descent(data, word2ind, N, V, C, num_iters)

Call gradient_descent
iters: 10 cost:  20.926946
iters: 20 cost:  20.431572
iters: 30 cost:  19.499856
iters: 40 cost:  19.447751
iters: 50 cost:  24.913858
iters: 60 cost:  30.860422
iters: 70 cost:  41.432009
iters: 80 cost:  52.029636
iters: 90 cost:  61.915792
iters: 100 cost:  67.170580
iters: 110 cost:  74.411181
iters: 120 cost:  88.073722
iters: 130 cost:  75.784061
iters: 140 cost:  114.903931
iters: 150 cost:  111.851899
iters: 160 cost:  95.805811
iters: 170 cost:  131.184741
iters: 180 cost:  141.219405
iters: 190 cost:  170.470644
iters: 200 cost:  170.023310
iters: 210 cost:  166.134576
iters: 220 cost:  168.716066
iters: 230 cost:  202.762959
iters: 240 cost:  201.496028
iters: 250 cost:  197.613939
iters: 260 cost:  184.757066
iters: 270 cost:  244.970770
iters: 280 cost:  218.092769
iters: 290 cost:  217.142600
iters: 300 cost:  228.613582
iters: 310 cost:  296.191329
iters: 320 cost:  270.879330
iters: 330 cost:  260.124359
iters: 340 cost:  275.910939
iters: 350 cost

C:\Users\vscerra\AppData\Local\Temp\ipykernel_19736\608088107.py:2: RuntimeWarning: overflow encountered in exp
  e_z = np.exp(z)
C:\Users\vscerra\AppData\Local\Temp\ipykernel_19736\608088107.py:5: RuntimeWarning: invalid value encountered in true_divide
  return e_z / sum_e_z
C:\Users\vscerra\AppData\Local\Temp\ipykernel_19736\119996856.py:3: RuntimeWarning: divide by zero encountered in log
  logprobs = np.multiply(np.log(yhat),y)
C:\Users\vscerra\AppData\Local\Temp\ipykernel_19736\119996856.py:3: RuntimeWarning: invalid value encountered in multiply
  logprobs = np.multiply(np.log(yhat),y)


iters: 390 cost:  nan
iters: 400 cost:  nan
iters: 410 cost:  nan
iters: 420 cost:  nan
iters: 430 cost:  nan
iters: 440 cost:  nan
iters: 450 cost:  nan
iters: 460 cost:  nan
iters: 470 cost:  nan
iters: 480 cost:  nan
iters: 490 cost:  nan
iters: 500 cost:  nan
iters: 510 cost:  nan
iters: 520 cost:  nan
iters: 530 cost:  nan
iters: 540 cost:  nan
iters: 550 cost:  nan
iters: 560 cost:  nan
iters: 570 cost:  nan
iters: 580 cost:  nan
iters: 590 cost:  nan
iters: 600 cost:  nan
iters: 610 cost:  nan
iters: 620 cost:  nan
iters: 630 cost:  nan
iters: 640 cost:  nan
iters: 650 cost:  nan
iters: 660 cost:  nan
iters: 670 cost:  nan
iters: 680 cost:  nan
iters: 690 cost:  nan
iters: 700 cost:  nan
iters: 710 cost:  nan
iters: 720 cost:  nan
iters: 730 cost:  nan
iters: 740 cost:  nan
iters: 750 cost:  nan
iters: 760 cost:  nan
iters: 770 cost:  nan
iters: 780 cost:  nan
iters: 790 cost:  nan
iters: 800 cost:  nan
iters: 810 cost:  nan
iters: 820 cost:  nan
iters: 830 cost:  nan
iters: 840

#### Cost function and gradient

The cost function used for logistic regression is the average of the log loss across all training examples:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) $$
* $m$ is the number of training examples
* $y^{(i)}$ is the actual label of training example 'i'.
* $h(z^{(i)})$ is the model's prediction for the training example 'i'.

The loss function for a single training example is
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* All the $h$ values are between 0 and 1, so the logs will be negative. That is the reason for the factor of -1 applied to the sum of the two loss terms.
